In [10]:
from torch.nn.modules.loss import _Loss
import torch.nn.functional as F
from torch import Tensor
import torch.utils.data as Data
import numpy as np
import torch
import datetime
from torch.autograd import Variable


class FlameLoss(_Loss):
    __constants__ = ['reduction']

    def __init__(self, size_average=None, reduce=None, reduction: str = 'sum') -> None:
        super(FlameLoss, self).__init__(size_average, reduce, reduction)

    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        return torch.sum((target + 1) * (input - target) ** 2)


def create_model(n_input, n_output, layers, device='cpu'):
    modules = []

    # add first hidden layer (and input layer)
    modules.append(torch.nn.Linear(n_input, layers[0]))
    modules.append(torch.nn.LeakyReLU())

    for i in range(1, len(layers)):
        modules.append(torch.nn.Linear(layers[i-1], layers[i]))
        modules.append(torch.nn.LeakyReLU())

    # add output layer
    modules.append(torch.nn.Linear(layers[-1], n_output))

    model = torch.nn.Sequential(*modules).to(device)
    return model


def train(model, X_train, y_train, X_test, y_test, device='cpu',
          n_epoch=100, lr=0.01, decayRate=0.96, BATCH_SIZE=64):
    
    # Process training data
    torch.manual_seed(1)    # reproducible
    train_dataset = Data.TensorDataset(X_train, y_train)
    loader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=False, )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    lr_decay = torch.optim.lr_scheduler.ExponentialLR(optimizer, decayRate)  # lr*gamma^step

    for epoch in range(n_epoch):
        for step, (batch_x, batch_y) in enumerate(loader):
            b_x = Variable(batch_x).to(device)
            b_y = Variable(batch_y).to(device)

            y_pred = model(b_x.float())
            loss = FlameLoss()(y_pred, b_y.float())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        lr_decay.step()

    # evaluate model:
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test.float())
        mseLoss = torch.nn.MSELoss(reduction='sum')(y_pred, y_test.float())

    y_err = (y_pred-y_test).cpu().data.numpy()
    std_err = np.abs(y_err).max(axis=0)   # CO_err, CO2_err
    return mseLoss, std_err[0], std_err[1]


def dl_modeling(X_train, y_train, X_test, y_test, layers, n_epoch=100):
    model = create_model(X_train.shape[1], y_train.shape[1], layers)
    return train(model, X_train, y_train, X_test, y_test, n_epoch=n_epoch)

## Create train/test data

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

save_dir = "/home/xiaopeng/Desktop/Flame_ann/"
df_all_norm = pd.read_pickle(save_dir + "data/train1_norm.pkl")  # reload unresampled
y = df_all_norm[["CO", "CO2"]]

X_train, X_test, y_train, y_test = train_test_split(df_all_norm, y, test_size=0.2, random_state=123)
X_train.to_pickle("data/train1_train.pkl")
X_test.to_pickle("data/train1_test.pkl")

## CPU training

In [57]:
df_train = pd.read_pickle(save_dir + "data/train1_train.pkl")  # reload unresampled
df_test = pd.read_pickle(save_dir + "data/train1_test.pkl")  # reload unresampled


X_train = torch.tensor(df_train[["CO+CO2", "N2"]].values, device='cpu')
y_train = torch.tensor(df_train[["CO", "CO2"]].values, device='cpu')
X_test = torch.tensor(df_test[["CO+CO2", "N2"]].values, device='cpu')
y_test = torch.tensor(df_test[["CO", "CO2"]].values, device='cpu')

nodes = [10, 10, 10]
loss, CO_err, CO2_err = dl_modeling(X_train, y_train, X_test, y_test, nodes, n_epoch=10)
node_str = '\t'.join([str(x) for x in nodes])
print(f"{node_str}\t{loss:.4f}\t{CO_err:.4f}\t{CO2_err:.4f}")

Start time = 2021-11-10 15:58:10.440823
10	10	10	7.0500	0.0946	0.1360


## Generate shell scripts

In [58]:
import argparse

def gen_2layer(save_dir, layer1, layer2, n_epoch=100, split_size=100):
    """
    :param save_dir: str. Path to save tuning results
    :param layer1: list. Number of nodes in the first layer
    :param layer2: list. Number of nodes in the second layer
    """
    counter = 0
    split = 1
    for i2 in range(len(layer2)):
        for i1 in range(len(layer1)):
            if counter >= split_size:   # split of 200 runs
                counter = 0
                split += 1
                if 'file' in locals():
                    file.close()
            if counter == 0:
                file = open(save_dir + f"tune_model/run_Flame_ann_tune_2layer_{split}.sh", "w")
                file.write("#!/bin/bash\n")
                file.write(f"data_dir={save_dir}data/\n")
                file.write(f"epochs={n_epoch}\n")

            file.write(f"python flame_ann_1cpu.py --data_dir $data_dir --epochs "
                       + f"$epochs --nodes {layer1[i1]} {layer2[i2]} "
                       + f">2l-{layer1[i1]}_{layer2[i2]}.out\n")
            counter += 1
    file.close()


def gen_3layer(save_dir, layer1, layer2, layer3, n_epoch=100, split_size=100):
    """
    :param save_dir: str. Path to save tuning results
    :param layer1: list. Number of nodes in the 1st layer
    :param layer2: list. Number of nodes in the 2nd layer
    :param layer3: list. Number of nodes in the 3rd layer
    :param n_epoch: int. epochs, default 100
    """
    counter = 0
    split = 1
    for i3 in range(len(layer3)):
        for i2 in range(len(layer2)):
            for i1 in range(len(layer1)):
                if counter >= split_size:   # each split of 200 runs
                    counter = 0
                    split += 1
                    if 'file' in locals():
                        file.close()
                if counter == 0:
                    file = open(save_dir + f"tune_model/run_Flame_ann_tune_3layer_{split}.sh", "w")
                    file.write("#!/bin/bash\n")
                    file.write(f"data_dir={save_dir}data/\n")
                    file.write(f"epochs={n_epoch}\n")

                file.write(f"python flame_ann_1cpu.py --data_dir $data_dir --epochs "
                           + f"$epochs --nodes {layer1[i1]} {layer2[i2]} {layer3[i3]} "
                           + f">3l-{layer1[i1]}_{layer2[i2]}_{layer3[i3]}.out\n")
                counter += 1
    file.close()


def gen_4layer(save_dir, layer1, layer2, layer3, layer4, n_epoch=100, split_size=100):
    """
    :param save_dir: str. Path to save tuning results
    :param layer1: list. Number of nodes in the 1st layer
    :param layer2: list. Number of nodes in the 2nd layer
    :param layer3: list. Number of nodes in the 3rd layer
    :param layer4: list. Number of nodes in the 4th layer
    """
    counter = 0
    split = 1
    for i4 in range(len(layer4)):
        for i3 in range(len(layer3)):
            for i2 in range(len(layer2)):
                for i1 in range(len(layer1)):
                    if counter >= split_size:   # each split of 200 runs
                        counter = 0
                        split += 1
                        if 'file' in locals():
                            file.close()
                    if counter == 0:
                        file = open(save_dir + f"tune_model/run_Flame_ann_tune_4layer_{split}.sh", "w")
                        file.write("#!/bin/bash\n")
                        file.write(f"data_dir={save_dir}data/\n")
                        file.write(f"epochs={n_epoch}\n")

                    file.write(f"python flame_ann_1cpu.py --data_dir $data_dir --epochs "
                               + f"$epochs --nodes {layer1[i1]} {layer2[i2]} {layer3[i3]} {layer4[i4]} "
                               + f">4l-{layer1[i1]}_{layer2[i2]}_{layer3[i3]}_{layer4[i4]}.out\n")
                    counter += 1
    file.close()


In [51]:
save_dir="/home/xiaopeng/Desktop/Flame_ann/"
                                                      # if not test
layer1 = [5, 10, 20, 40, 80, 160]
layer2 = [5, 10, 20, 40, 80]
layer3 = [5, 10, 20, 40, 80]
layer4 = [5, 10, 20, 40]

gen_2layer(folder_path, layer1, layer2, n_epoch=100)
gen_3layer(save_dir, layer1, layer2, layer3, n_epoch=100)
gen_4layer(save_dir, layer1, layer2, layer3, layer4, n_epoch=100)

### Contents of sbatch scripts (to run on ibex)
Prefix contents

```python
#!/bin/bash
#SBATCH --time=4:00:00
#SBATCH --cpus-per-task=60
#SBATCH --mem=64G
#SBATCH --partition=batch
#SBATCH --job-name=2layer
#SBATCH --output=./%x-%j-slurm.out
#SBATCH --error=./%x-%j-slurm.err

start_time=$(date +"%T")
echo "Start time: $start_time"

data_dir=/home/xux/work/Flame_ann/data/
epochs=100
```

Suffix contents
```python
finish_time=$(date +"%T")
echo "Finish time: $finish_time"
```